In [1]:

from connectors.postgres_connector import PostgresConnector
from feeds.postgres_feed import PostgresFeed

import dedupe

pg_conn = PostgresConnector(db='defaultdb', username='jameyc', password='UXZSXXXSFZeU8XKw', host='silobuster-db-do-user-12298230-0.b.db.ondigitalocean.com', port=25060)
select_qry = "SELECT id, name, address FROM what_location WHERE address is not null and not address = '' and name is not null and not name = ''"
# select_qry2 = "SELECT id, name, address FROM what_location"


raw_def = [
    {"field": "name", "type": "String"},
    {"field": "address", "type": "String"},
    
]


pg_feed = PostgresFeed.from_manual(connector=pg_conn, query=select_qry, column_definition=raw_def, primary_key='id')

# select_qry = "SELECT name, address FROM what_location WHERE address is not null and not address = '' and name is not null and not name = ''"
deduper = dedupe.Dedupe(pg_feed.variable_definition)



In [2]:
deduper.prepare_training(pg_feed.formatted_data, sample_size=100000, blocked_proportion=1)

In [3]:
distinct_pairs = []
while True:
    try:
        pair = deduper.uncertain_pairs()
        distinct_pairs.extend(pair)
    except IndexError as e:
        break
    except Exception as e:
        raise

deduper.mark_pairs({
    "match": [],
    "distinct": distinct_pairs
})

In [4]:
print (distinct_pairs)

[({'name': 'Sea Mar Bellingham Dental Clinic', 'address': '4455 Cordata Pkwy.'}, {'name': 'Sea Mar Bellingham Medical Clinic', 'address': '4455 Cordata Pkwy.'}), ({'name': "Unity Care NW Bellingham Medical and Children's Dental Services", 'address': '220 Unity St.'}, {'name': 'Unity Care NW Bellingham Pharmacy', 'address': '218 Unity St.'}), ({'name': 'Base Camp', 'address': '1530 Cornwall Ave.'}, {'name': 'Unity Care NW Bellingham Adult Dental, Behavioral Health, Donations, and Administration', 'address': '1616 Cornwall Ave.'}), ({'name': 'Sumas Community Senior Center', 'address': '461 2nd St.'}, {'name': 'Sumas Library', 'address': '461 2nd St.'}), ({'name': 'PeaceHealth at Cordata Main', 'address': '4545 Cordata Pkwy.'}, {'name': 'PeaceHealth OB/GYN at Cordata South', 'address': '4465 Cordata Pkwy.'}), ({'name': 'CADA', 'address': '275 SE Pioneer Way '}, {'name': 'DSHS Island County', 'address': '275 SE Pioneer Way '}), ({'name': 'CADA', 'address': '275 SE Pioneer Way '}, {'name': 

In [5]:
deduper.train()

ValueError: 
All the 35 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
35 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1183, in fit
    raise ValueError(
ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0
